In [1]:
import sys 
import os 
sys.path.insert(0, os.path.dirname(os.getcwd()))

In [2]:
from utils import get_connection
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [3]:
con = get_connection()

connecting to rm-uf6lfxk83h7onjmwvjo.mysql.rds.aliyuncs.com3306/


In [4]:
pd.read_sql("show databases ",con)


,Database
0,douyin
1,information_schema
2,mysql
3,performance_schema
4,validation
5,xiaohongshu


In [5]:
df_selective_mapping = pd.read_sql("SELECT * FROM validation.tag_group_text_topic_validation" ,con)
df_selective_mapping = df_selective_mapping.rename(columns= {'tag_old': 'tag_value', 'tag_new':'class_name'})

In [6]:
df_selective_mapping

,tag_value,class_name
0,旅行推荐,旅行
1,产品评价,其他
2,产品推荐,广告与推广
3,餐饮体验,美食分享与推荐
4,自制饮品,烹饪与食谱
...,...,...
730,家庭烹饪,烹饪与食谱
731,农家土特产,美食分享与推荐
732,家庭分享/孝心表达,家庭日常
733,旅游+美食体验,旅行


In [7]:
dict_sql = """
select tag_category, tag_value, keyword
from (
SELECT *
FROM validation.`tag_kwd_text_topic_prompt_v2`
) a 
"""


In [8]:

df_dictionary = pd.read_sql(dict_sql, con = con)
print(df_dictionary.shape)
# df_dictionary = df_dictionary.merge(df_selective_mapping, on = 'tag_value')
df_dictionary['kw_len'] = df_dictionary['keyword'].apply(len)
df_dictionary = df_dictionary.query("kw_len <= 8 & kw_len > 1")
print("filtered dictionary:" , df_dictionary.shape)

# df_dictionary = (
#     df_dictionary
#     .groupby(['tag_category', 'class_name'])
#     .apply(lambda x: list(set(x['keyword'])))
#     .rename(index = 'kw_ls')
#     .reset_index()
# )

# print("ttl kw:", df_dictionary['kw_ls'].str.len().sum())

df_dictionary.head()

(25594, 3)
filtered dictionary: (22773, 4)


,tag_category,tag_value,keyword,kw_len
0,文本主题,社交互动,女儿的开心,5
1,文本主题,社交互动,和伙伴一起去玩,7
2,文本主题,社交互动,挑自己喜欢的零食,8
3,文本主题,社交互动,和妈妈分享,5
7,文本主题,运动健身,俯卧撑,3


In [22]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')


In [23]:
keyword_ls = df_dictionary['keyword'].unique().tolist()

embeddings = model.encode(keyword_ls)

In [24]:
import numpy as np 

embeddings_arr = np.array(embeddings.tolist())

from sklearn.manifold import TSNE


x_tsne = TSNE(n_components=2, learning_rate='auto',init='random', perplexity=50).fit_transform(embeddings_arr)

In [25]:
df_kw_embeddings = pd.DataFrame({'keyword': keyword_ls, 'embedding':embeddings_arr.tolist(), 'tsne1': x_tsne[:,0], 'tsne2': x_tsne[:,1]})

In [90]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca.fit(embeddings_arr)
x_pca = pca.transform(embeddings_arr)
df_kw_embeddings['pca1'] = x_pca[:,0]
df_kw_embeddings['pca2'] = x_pca[:,1]

In [91]:
df_dictionary_embed = df_dictionary.merge(df_kw_embeddings, on = 'keyword', how = 'left')

In [92]:
selective_tags = ['健康与养生',
 '学习与教育',
 '生活感悟',
 '运动健身',
 '烹饪与食谱',
 '旅行',
 '家庭日常',
 '美食分享与推荐',
 '其他',
 '儿童饮食与健康',
 '节日庆祝',
 '体重管理',
 '户外活动',
 '食物评测',
 '广告与推广',
 '怀旧与回忆',
 '孕期饮食与健康',
 '校园生活',
 '社交互动',
 '健康与饮食',
 '日常记录']

df_dictionary_embed['tag_value'] = df_dictionary_embed['tag_value'].apply(lambda x: x if x in selective_tags else "其他")

In [93]:
import plotly.express as px
import numpy as np



data = df_dictionary_embed

color_ls = px.colors.qualitative.Dark24

class_name_ls = df_dictionary_embed['tag_value'].unique().tolist()


color_dict = {t:c  for t, c in zip(class_name_ls,color_ls)}


fig = px.scatter(df_dictionary_embed, x="pca1", y="pca2", color='tag_value', hover_name='keyword',hover_data='tag_value')
fig.update_layout(template = 'simple_white', width = 700)



In [75]:
import sklearn 
sklearn.__version__

'1.4.2'

In [98]:
from sklearn.neighbors import LocalOutlierFactor
import numpy as np

def scoring(x, **kwargs):
    X_tag =np.array(x['embedding'].tolist())
    lof = LocalOutlierFactor()  # 可以根据具体情况调整contamination参数
    lof.fit(X_tag)
    outlier_scores = lof.negative_outlier_factor_
    outlier_scores_nor = np.exp(outlier_scores)/np.exp(outlier_scores).sum()

    return outlier_scores_nor



df_dictionary_embed_score = pd.DataFrame()

from tqdm import tqdm
for tag_value, df_dictionary_embed_tag in tqdm(df_dictionary_embed.groupby('tag_value')):
    df_dictionary_embed_tag['score'] = scoring(df_dictionary_embed_tag)
    df_dictionary_embed_score = pd.concat([df_dictionary_embed_score, df_dictionary_embed_tag, ], axis = 0)
    
df_dictionary_embed_score

  0%|          | 0/21 [00:00<?, ?it/s]

100%|██████████| 21/21 [00:01<00:00, 12.07it/s]


,tag_category,tag_value,keyword,kw_len,embedding,tsne1,tsne2,pca1,pca2,score
272,文本主题,体重管理,身高158,5,"[0.25168919563293457, 0.4568338096141815, -0.0...",-64.808800,-6.780519,-0.955509,-0.303214,0.000916
273,文本主题,体重管理,今日47.95,7,"[0.02312518283724785, 0.004103669431060553, -0...",-71.489395,-13.427218,-0.631440,-0.483863,0.000831
274,文本主题,体重管理,瘦身日记,4,"[0.10391997545957565, 0.23133674263954163, -0....",-1.883632,-70.082138,-0.638046,-0.514125,0.001150
275,文本主题,体重管理,减肥减脂,4,"[0.01258939504623413, 0.07157845795154572, -0....",-59.856541,31.363527,0.924511,-1.760813,0.000000
276,文本主题,体重管理,早餐,2,"[-0.21518364548683167, -0.19477519392967224, 0...",-20.768791,80.695206,2.805239,-0.306618,0.001549
...,...,...,...,...,...,...,...,...,...,...
21667,文本主题,食物评测,朋友的评价,5,"[0.03651963174343109, -0.018362490460276604, -...",-18.011354,-42.203197,-0.526817,-0.321908,0.006263
21668,文本主题,食物评测,味道疑问,4,"[0.12025491148233414, -0.11232540756464005, 0....",-27.645399,40.976337,0.918896,-0.124654,0.007278
21669,文本主题,食物评测,制作技巧讨论,6,"[0.07652824372053146, 0.015844101086258888, -0...",-4.644191,-34.890556,-0.524204,-0.350101,0.005876
21670,文本主题,食物评测,零食分享,4,"[-0.1715155690908432, -0.27274540066719055, 0....",-16.059919,57.460442,1.976298,0.124010,0.005748


In [116]:
import plotly.io as pio

In [126]:
html_strings = "" 


In [127]:
df_dictionary_embed_tag = df_dictionary_embed[df_dictionary_embed['tag_value'] == '儿童饮食与健康']




import numpy as np
from sklearn.neighbors import KernelDensity

# kde = KernelDensity(bandwidth=1.2, kernel='gaussian')

# kde.fit(X_tag)
from sklearn.neighbors import LocalOutlierFactor
import numpy as np

lof = LocalOutlierFactor()  # 可以根据具体情况调整contamination参数
lof.fit(X_tag)
outlier_scores = lof.negative_outlier_factor_
outlier_scores_nor = np.exp(outlier_scores)/np.exp(outlier_scores).sum()

df_dictionary_embed_tag['score'] = outlier_scores_nor


df_dictionary_embed_tag['score'] = outlier_scores_nor

fig = px.scatter(df_dictionary_embed_tag, x="pca1", y="pca2", color = 'score', hover_name='keyword',hover_data=['tag_value','score'])
fig.update_layout(template = 'simple_white', height = 700, width = 800, title = '儿童饮食与健康')
    
html_strings += pio.to_html(fig, full_html=False)

fig = px.scatter(df_dictionary_embed_tag, x="pca1", y="pca2", hover_name='keyword',hover_data=['tag_value','score'])
fig.update_layout(template = 'simple_white', height = 700, width = 800, title = '儿童饮食与健康')
for d in fig.data:
      d['marker']['color'] = 'white'

for i, data_ in df_dictionary_embed_tag.iterrows():
    fig.add_annotation(
        font = dict(size = 8),
        xref="x",
        yref="y",
        x=data_['pca1'],
        y=data_['pca2'],
        text=data_['keyword'],
        # arrowhead=0,
        showarrow=False
    )
    
# fig.show()
html_strings += pio.to_html(fig, full_html=False)

In [ ]:
df_dictionary_embed_score

In [128]:
with open('data/儿童饮食与健康_scoring.html', 'w') as f:
    f.write(html_strings)

In [105]:
import pickle
with open('df_dictionary_embed_score.pickle', 'wb') as handle:
    pickle.dump(df_dictionary_embed_score,handle)


In [173]:
def _exact_matching(x, kw_ls, tag_name):
    tag:str = None
    kw_matched = []
    for kw in kw_ls:
        if kw in x:
            tag = tag_name
            kw_matched.append(kw)
            # break
    return tag,kw_matched
    
def tag_data_one_category_score(content, df_dictionary:pd.DataFrame, kw_col = 'keyword', tag_value_col = 'tag_value', score_col = 'score'):
    tag_value_ls = []

    for tag_value, _dictionary in df_dictionary.groupby(tag_value_col):
        kw_ls = _dictionary[kw_col].unique()

        tag, kw_matched = _exact_matching(content, kw_ls, tag_value)
        
        score_dict = _dictionary.set_index(kw_col)[score_col].to_dict()
        kw_matched_score = [score_dict[k] for k in kw_matched] if tag else []

        if tag:
            tag_value_ls.append({tag: {'score': np.sum(kw_matched_score), 'keywords': {k:s for k,s in zip(kw_matched,kw_matched_score)}}})
    return tag_value_ls
        
def tag_data_multi_category_score(data, df_dictionary, categories, category_col = 'tag_category', content_col = 'content', kw_col = 'keyword', tag_value_col = 'tag_value'):
    content = data[content_col]
    rst = []
    for cate in categories:
        _df_dictionary = df_dictionary[df_dictionary[category_col] == cate]
        tags = tag_data_one_category_score(content, _df_dictionary, kw_col, tag_value_col)
        rst.append(tags)
    return rst

def tag_data_score(df_dictionary, data,category_col = 'tag_category', content_col = 'content', kw_col = 'keyword', tag_value_col = 'tag_value'):
    data = data.copy()
    categories = df_dictionary['tag_category'].unique().tolist()
    args = dict(category_col =category_col, content_col = content_col, kw_col = kw_col, tag_value_col = tag_value_col)
    data[categories] = data.progress_apply(tag_data_multi_category_score, axis = 1, result_type ='expand',df_dictionary = df_dictionary, categories= categories, **args)
    return data

In [152]:
from tqdm import tqdm 
tqdm.pandas()

In [138]:
data = pd.read_csv('data/validation_3000.csv')
data.head()

,id,content,output,text_topic
0,e04592c0da33650d9c837464dca6c706|b9393c69e2e6d...,一拖二的周六，一早起床，勿勿忙忙准备好小娃需要看病的资料，再准备我需要摆摊的工具，大娃补课催...,"{'食品类别': '未知', '是否为零售食品': '否', '文本主题': '日常生活挑战...",日常生活挑战
1,18336dc645f26e3c3944e8c47c1c9831|d035d1c7e67d6...,津铺子的这个0卡果冻回购很多次了，一共四种口味：白桃、青提、荔枝和芒果。四个口味都还不错，口...,"{'食品类别': '蜜饯果干', '是否为零售食品': '是', '文本主题': '产品推荐...",产品推荐
2,b9240d65115b682962187bac31a5a14d|e64571aeb3fbf...,话梅柠檬茶！建议女孩子都把奶茶换成它！ 全网安利的咸柠茶！我能喝一整个夏天！真的好喝到原地转...,"{'食品类别': '未知', '是否为零售食品': '是', '文本主题': '饮品推荐',...",饮品推荐
3,a0e6ebe573ea14303727123a5abbef3e|de754b5d37397...,随手买的香辣鱼尾也太好吃了吧！鱼肉很有嚼劲，麻麻辣辣的也太过瘾了，平时追剧或者嘴馋了来一包，...,"{'食品类别': '肉脯卤味', '是否为零售食品': '是', '文本主题': '食品体验...",食品体验
4,d3fc3728a8c3e13ae5988aa15e4bb0fd|320b8f65d7782...,#踏春赏花最美的景点推荐 #感受大自然的气息和美景 #旅行推荐官 #抖音带你去赏花 #春暖花...,"{'食品类别': '未知', '是否为零售食品': '否', '文本主题': '旅游推荐',...",旅游推荐


In [175]:
tagged_data = tag_data_score(df_dictionary_embed_score, data, tag_value_col='tag_value')
tagged_data


100%|██████████| 3000/3000 [00:53<00:00, 55.65it/s]


,id,content,output,text_topic,文本主题
0,e04592c0da33650d9c837464dca6c706|b9393c69e2e6d...,一拖二的周六，一早起床，勿勿忙忙准备好小娃需要看病的资料，再准备我需要摆摊的工具，大娃补课催...,"{'食品类别': '未知', '是否为零售食品': '否', '文本主题': '日常生活挑战...",日常生活挑战,"[{'体重管理': {'score': 0.0033671740349967547, 'ke..."
1,18336dc645f26e3c3944e8c47c1c9831|d035d1c7e67d6...,津铺子的这个0卡果冻回购很多次了，一共四种口味：白桃、青提、荔枝和芒果。四个口味都还不错，口...,"{'食品类别': '蜜饯果干', '是否为零售食品': '是', '文本主题': '产品推荐...",产品推荐,"[{'体重管理': {'score': 0.001527535237278284, 'key..."
2,b9240d65115b682962187bac31a5a14d|e64571aeb3fbf...,话梅柠檬茶！建议女孩子都把奶茶换成它！ 全网安利的咸柠茶！我能喝一整个夏天！真的好喝到原地转...,"{'食品类别': '未知', '是否为零售食品': '是', '文本主题': '饮品推荐',...",饮品推荐,"[{'体重管理': {'score': 0.000979063920091003, 'key..."
3,a0e6ebe573ea14303727123a5abbef3e|de754b5d37397...,随手买的香辣鱼尾也太好吃了吧！鱼肉很有嚼劲，麻麻辣辣的也太过瘾了，平时追剧或者嘴馋了来一包，...,"{'食品类别': '肉脯卤味', '是否为零售食品': '是', '文本主题': '食品体验...",食品体验,"[{'体重管理': {'score': 0.001527535237278284, 'key..."
4,d3fc3728a8c3e13ae5988aa15e4bb0fd|320b8f65d7782...,#踏春赏花最美的景点推荐 #感受大自然的气息和美景 #旅行推荐官 #抖音带你去赏花 #春暖花...,"{'食品类别': '未知', '是否为零售食品': '否', '文本主题': '旅游推荐',...",旅游推荐,"[{'体重管理': {'score': 0.0012007433419263702, 'ke..."
...,...,...,...,...,...
2995,594ac51b1657f4738586e8ae24552b52|2be16c8d4707b...,#好吃又便宜的网红零食推荐 发现一款超好吃的小零食，就是这款九块九三十包的酥牛排，还是盐津铺...,"{'食品类别': '肉脯卤味', '是否为零售食品': '是', '文本主题': '零食推荐...",零食推荐,"[{'体重管理': {'score': 0.002634477824155019, 'key..."
2996,357b96bad7f1fa478a787ae85785d175|6298eda597a46...,救命这个燕麦酥真的不要太好吃！\n这款燕麦巧克力酥不含反式脂肪，外皮包裹巧克力\n再配上饱满...,"{'食品类别': '未知', '是否为零售食品': '是', '文本主题': '零食推荐',...",零食推荐,"[{'体重管理': {'score': 0.003532816325449019, 'key..."
2997,956e86edcfd1f8201401df05c29181f3|846599ba7abdd...,尊敬的顾客朋友们\n🔔加一加水果零食明天4.18举行年会\n🙋🏻‍♀️全体员工参加⚠️闭店一...,"{'食品类别': '未知', '是否为零售食品': '是', '文本主题': '商业通知',...",商业通知,"[{'体重管理': {'score': 0.0022859928391844603, 'ke..."
2998,7a6c58420d0aab1b5fc6f51a583503fe|9451fadf499ed...,#女大学生 #淄博烧烤 去淄博喽，八大局去了，紫米饼没吃，炒锅饼没吃，牛奶棒没吃，小蛋糕没吃...,"{'食品类别': '未知', '是否为零售食品': '否', '文本主题': '旅行体验',...",旅行体验,"[{'体重管理': {'score': 0.001654673152136547, 'key..."


In [189]:
x = tagged_data['文本主题'].values[0]

def max_score(x):
    d = {}
    for a in x:
        d.update(a)
        
    kw_sort = sorted(d.keys(), key =  lambda x: d[x]['score'])
    
    return kw_sort[-1] if len(kw_sort) else []

max_score(x)

tagged_data['by_score'] = tagged_data['文本主题'].apply(max_score)

In [190]:
tagged_data.to_csv('kwyword_score_on_3000.csv', index=False)

In [186]:
x

[{'体重管理': {'score': 0.0033671740349967547,
   'keywords': {'记录': 0.0010765925463304975,
    '第一天': 0.0010898381467398868,
    '烤肠': 0.0012007433419263702}}},
 {'其他': {'score': 0.0009430252539550953,
   'keywords': {'生活': 0.00014911456993932736,
    '摆摊': 0.00016483027452245772,
    '努力': 0.00010755267041094657,
    '第一天': 7.401891843095604e-05,
    '记录摆摊生活': 0.00012684903723515458,
    '烤肠': 0.00016825529306719694,
    '每天': 0.00015240449034905615}}},
 {'家庭日常': {'score': 0.02568701172971338,
   'keywords': {'生活': 0.0024846860714609234,
    '摆摊': 0.0027975523502543668,
    '第一天': 0.0024114586480179193,
    '一拖二': 0.002998980040885452,
    '烤肠': 0.002927909724371525,
    '小娃': 0.003495881037708807,
    '看病': 0.003320169319439349,
    '大娃补课': 0.003032901743909356,
    '电动车': 0.0022174727936656847}}},
 {'日常记录': {'score': 0.010695792055901378,
   'keywords': {'摆摊生活': 0.003928086148382451,
    '记录': 0.0032008611877711933,
    '摆摊': 0.003566844719747733}}},
 {'生活感悟': {'score': 0.0063373795954

In [184]:
x

[{'体重管理': {'score': 0.0033671740349967547,
   'keywords': {'记录': 0.0010765925463304975,
    '第一天': 0.0010898381467398868,
    '烤肠': 0.0012007433419263702}}},
 {'其他': {'score': 0.0009430252539550953,
   'keywords': {'生活': 0.00014911456993932736,
    '摆摊': 0.00016483027452245772,
    '努力': 0.00010755267041094657,
    '第一天': 7.401891843095604e-05,
    '记录摆摊生活': 0.00012684903723515458,
    '烤肠': 0.00016825529306719694,
    '每天': 0.00015240449034905615}}},
 {'家庭日常': {'score': 0.02568701172971338,
   'keywords': {'生活': 0.0024846860714609234,
    '摆摊': 0.0027975523502543668,
    '第一天': 0.0024114586480179193,
    '一拖二': 0.002998980040885452,
    '烤肠': 0.002927909724371525,
    '小娃': 0.003495881037708807,
    '看病': 0.003320169319439349,
    '大娃补课': 0.003032901743909356,
    '电动车': 0.0022174727936656847}}},
 {'日常记录': {'score': 0.010695792055901378,
   'keywords': {'摆摊生活': 0.003928086148382451,
    '记录': 0.0032008611877711933,
    '摆摊': 0.003566844719747733}}},
 {'生活感悟': {'score': 0.0063373795954

In [228]:
tagged_data['文本主题Max'] = \
    tagged_data['文本主题'].apply(
        lambda x: [list(i.keys())[0] for i in x][np.array([len(i[list(i.keys())[0]]) for i in x]).argmax()] if len(x) else x
    )


In [229]:
tagged_data.to_csv('data/3000dictionary_on_3000.csv', index=False)

In [153]:
tagged_data['文本主题'].apply(lambda x: pd.Series(x.split(',')).value_counts().to_dict())

0       {'体重管理': 1, '健康与养生': 1, '健康与饮食': 1, '其他': 1, '...
1       {'体重管理': 1, '旅行': 1, '运动健身': 1, '节日庆祝': 1, '美食...
2       {'体重管理': 1, '健康与养生': 1, '健康与饮食': 1, '儿童饮食与健康':...
3       {'体重管理': 1, '健康与养生': 1, '运动健身': 1, '节日庆祝': 1, ...
4       {'体重管理': 1, '旅行': 1, '运动健身': 1, '节日庆祝': 1, '美食...
                              ...                        
2995    {'体重管理': 1, '健康与养生': 1, '运动健身': 1, '节日庆祝': 1, ...
2996    {'体重管理': 1, '旅行': 1, '运动健身': 1, '节日庆祝': 1, '美食...
2997    {'体重管理': 1, '旅行': 1, '运动健身': 1, '节日庆祝': 1, '美食...
2998    {'体重管理': 1, '日常记录': 1, '运动健身': 1, '节日庆祝': 1, '...
2999    {'体重管理': 1, '旅行': 1, '运动健身': 1, '节日庆祝': 1, '美食...
Name: 文本主题, Length: 3000, dtype: object

In [115]:
pd.read_sql('show tables', get_connection(database= 'validation'))

connecting to rm-uf6lfxk83h7onjmwvjo.mysql.rds.aliyuncs.com3306/validation


,Tables_in_validation
0,tag_group_text_topic
1,tag_group_text_topic_validation
2,tag_kwd_character_type
3,tag_kwd_emotional_atmosphere
4,tag_kwd_food_category
5,tag_kwd_food_usage
6,tag_kwd_is_retail_food
7,tag_kwd_macro_occasion
8,tag_kwd_micro_occasion
9,tag_kwd_painpoint


In [125]:
from io import StringIO
import json

data = pd.read_csv(StringIO(text), sep= '\t')

data['output'] = data['output'].apply(lambda x: json.loads(x.replace("'", '"')))

In [127]:
data['text_topic'] = data['output'].apply(lambda x: x['文本主题'])

In [132]:
data.to_csv('data/validation_3000.csv', index=False)

In [131]:
data

,id,content,output,text_topic
0,e04592c0da33650d9c837464dca6c706|b9393c69e2e6d...,一拖二的周六，一早起床，勿勿忙忙准备好小娃需要看病的资料，再准备我需要摆摊的工具，大娃补课催...,"{'食品类别': '未知', '是否为零售食品': '否', '文本主题': '日常生活挑战...",日常生活挑战
1,18336dc645f26e3c3944e8c47c1c9831|d035d1c7e67d6...,津铺子的这个0卡果冻回购很多次了，一共四种口味：白桃、青提、荔枝和芒果。四个口味都还不错，口...,"{'食品类别': '蜜饯果干', '是否为零售食品': '是', '文本主题': '产品推荐...",产品推荐
2,b9240d65115b682962187bac31a5a14d|e64571aeb3fbf...,话梅柠檬茶！建议女孩子都把奶茶换成它！ 全网安利的咸柠茶！我能喝一整个夏天！真的好喝到原地转...,"{'食品类别': '未知', '是否为零售食品': '是', '文本主题': '饮品推荐',...",饮品推荐
3,a0e6ebe573ea14303727123a5abbef3e|de754b5d37397...,随手买的香辣鱼尾也太好吃了吧！鱼肉很有嚼劲，麻麻辣辣的也太过瘾了，平时追剧或者嘴馋了来一包，...,"{'食品类别': '肉脯卤味', '是否为零售食品': '是', '文本主题': '食品体验...",食品体验
4,d3fc3728a8c3e13ae5988aa15e4bb0fd|320b8f65d7782...,#踏春赏花最美的景点推荐 #感受大自然的气息和美景 #旅行推荐官 #抖音带你去赏花 #春暖花...,"{'食品类别': '未知', '是否为零售食品': '否', '文本主题': '旅游推荐',...",旅游推荐
...,...,...,...,...
2995,594ac51b1657f4738586e8ae24552b52|2be16c8d4707b...,#好吃又便宜的网红零食推荐 发现一款超好吃的小零食，就是这款九块九三十包的酥牛排，还是盐津铺...,"{'食品类别': '肉脯卤味', '是否为零售食品': '是', '文本主题': '零食推荐...",零食推荐
2996,357b96bad7f1fa478a787ae85785d175|6298eda597a46...,救命这个燕麦酥真的不要太好吃！\n这款燕麦巧克力酥不含反式脂肪，外皮包裹巧克力\n再配上饱满...,"{'食品类别': '未知', '是否为零售食品': '是', '文本主题': '零食推荐',...",零食推荐
2997,956e86edcfd1f8201401df05c29181f3|846599ba7abdd...,尊敬的顾客朋友们\n🔔加一加水果零食明天4.18举行年会\n🙋🏻‍♀️全体员工参加⚠️闭店一...,"{'食品类别': '未知', '是否为零售食品': '是', '文本主题': '商业通知',...",商业通知
2998,7a6c58420d0aab1b5fc6f51a583503fe|9451fadf499ed...,#女大学生 #淄博烧烤 去淄博喽，八大局去了，紫米饼没吃，炒锅饼没吃，牛奶棒没吃，小蛋糕没吃...,"{'食品类别': '未知', '是否为零售食品': '否', '文本主题': '旅行体验',...",旅行体验


In [86]:
con = get_connection()
n = pd.read_sql(f"select count(*) count from ({data_sql}) a", con = con)['count'].values[0]
chunksize = 100
step = int(n/chunksize)
n
for data in tqdm(pd.read_sql(data_sql, con = con, chunksize = chunksize), total = step):
    data = tag_data(df_dictionary, data)
    data[['id', 'ds', *df_dictionary['tag_category'].unique().tolist() ]].to_csv('data/tagged_validation.csv', mode = 'a', index=False)

connecting to rm-uf6lfxk83h7onjmwvjo.mysql.rds.aliyuncs.com3306/


  0%|          | 0/28 [00:00<?, ?it/s]


KeyError: 'kw_ls'

In [11]:
import multiprocessing as mp
from multiprocessing import Pool
con = get_connection(database='xiaohongshu')
n_process = mp.cpu_count()

def func(data):
    return tag_data(df_dictionary, data)
    
n = pd.read_sql(f"select count(*) count from ({data_sql}) a", con = con)['count'].values[0]
chunksize = 10
step = int(n/chunksize)
iterator = pd.read_sql(data_sql, con = con, chunksize = chunksize)

if __name__  == '__main__':
    pool = Pool(n_process)
    for d in tqdm(pool.imap_unordered(func, iterator), total = step):
        d[['id', 'ds', *df_dictionary['tag_category'].unique().tolist() ]].to_csv('data/tagged.csv', mode = 'a', index=False)

connecting to rm-uf6lfxk83h7onjmwvjo.mysql.rds.aliyuncs.com3306/xiaohongshu


  0%|          | 0/587164 [00:00<?, ?it/s]

In [28]:
data.loc[:1,'content'].values[0]

'先说万豪白金不烂大街哦[萌萌哒R]北上广江浙一带商旅人士多，三亚旅游的人多，可能大家看到的现象就觉得万豪白金烂大街，二三线城市还是非常非常好用的，具体权益和获取办法可以看我前面笔记[暗中观察R]\n\t\n[举手R]今天给大家讲一个小众且高级的酒店联盟：GHA\n大名鼎鼎的尼依格罗、三亚嘉佩乐就是这个联盟。\n划重点：几百块钱能升级的[二R]年有效期酒店🔝会籍，只有GHA钛金！（之前bug渠道已修复）\n\t\n[举手R]GHA钛金会员权益：\n1：客房连升两级，包括行政房或套房，房型差价1k+\n2：大部分酒店视情况供应免费双早和行政待遇，需要提前问酒店\n3：提前至11点入住，次日可延迟至16点退房。\n4：欢迎水果和点心\n5：免费游泳池健身房\n\t\n而且它秒升！秒升诶！[赞R]着急入住的快冲[赞R]'

In [ ]:
['健康与养生',
 '学习与教育',
 '生活感悟',
 '运动健身',
 '烹饪与食谱',
 '旅行',
 '家庭日常',
 '美食分享与推荐',
 '其他',
 '儿童饮食与健康',
 '节日庆祝',
 '体重管理',
 '户外活动',
 '食物评测',
 '广告与推广',
 '怀旧与回忆',
 '孕期饮食与健康',
 '校园生活',
 '社交互动',
 '健康与饮食',
 '日常记录']